<a href="https://colab.research.google.com/github/mralamdari/NLP-Text_Classification/blob/main/NLP_Text_Classification_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget
!pip install ktrain

In [2]:
import os
import sys
import wget
import nltk
import numpy
import ktrain
import tarfile
import warnings
from ktrain import text
import tensorflow as tf
from zipfile import ZipFile

warnings.filterwarnings('ignore')

# Data
###IMDB Dataset

In [4]:
!wget -P DATAPATH http://nlp.stanford.edu/data/glove.6B.zip
!unzip DATAPATH/glove.6B.zip -d DATAPATH/glove.6B

!wget -P DATAPATH http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xvf DATAPATH/aclImdb_v1.tar.gz -C DATAPATH
BASE_DIR = 'Data'

In [7]:
BASE_DIR = '/content/DATAPATH'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
TRAIN_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/train')
TEST_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/test')

In [8]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [13]:
def get_data(data_dir):
  texts = []
  labels_index = {'pos': 1, 'neg': 0}
  labels = []
  for name in sorted(os.listdir(data_dir)):
    path = os.path.join(data_dir, name)
    if os.path.isdir(path):
      if name=='pos' or name=='neg':
        label_id = labels_index[name]
        for fname in sorted(os.listdir(path)):
          fpath = os.path.join(path, fname)
          text = open(fpath, encoding='utf8').read()
          texts.append(text)
          labels.append(label_id)
  return texts, labels

In [14]:
train_texts, train_labels = get_data(TRAIN_DATA_DIR)
test_texts, test_labels = get_data(TEST_DATA_DIR)

In [16]:
train_texts[20]

"The characters are unlikeable and the script is awful. It's a waste of the talents of Deneuve and Auteuil."

#BERT Sentiment Classification
## IMDB ktrain